In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import re
from collections import defaultdict

SEQUENCE_LEN = 20
EMBEDDING_DIM = 100
RNN_LAYER_DIM = 30
EPOCHS = 50
BATCH_SIZE = 8


class RnnModel:
    def __init__(self, seq_len, vocab_size, h1_dim, embedding_dim) -> None:
        self.vocab_size = vocab_size
        self._model = self._build_model(seq_len, vocab_size, h1_dim, embedding_dim)

    def _build_model(self, seq_len, vocab_size, h1_dim, embedding_dim):
        input = keras.Input(shape=(seq_len,), dtype="int32")
        embedding = keras.layers.Embedding(vocab_size, embedding_dim)(input)
        rnn_layer = keras.layers.SimpleRNN(h1_dim)(embedding)
        output = keras.layers.Dense(vocab_size, activation="softmax")(rnn_layer)
        model = keras.Model(inputs=input, outputs=output)
        model.compile(
            optimizer=keras.optimizers.Adam(),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

    def forward(self, input):
        return self._model.predict(input)

    def fit(self, input, target, batch_size=16, epochs=10):
        history = self._model.fit(
            input, target, batch_size=batch_size, epochs=epochs, verbose=1
        )
        return history.history

    def get_weights(self):
        return self._model.get_weights()

    def set_weights(self, weights):
        self._model.set_weights(weights)

    def generate(self, seed_tokens, idx2word, max_length=50):
        """
        seed_tokens: list[int] - starting token IDs, length must be SEQUENCE_LEN
        idx2word: dict[int, str] - mapping from token IDs to words
        """
        generated = list(seed_tokens)

        for _ in range(max_length):
            x_input = np.array([generated[-SEQUENCE_LEN:]])
            pred_probs = self._model.predict(x_input, verbose=0)[0]
            next_id = np.argmax(pred_probs)

            generated.append(next_id)

            if idx2word[next_id] == "<#END>":
                break

        return " ".join(idx2word[idx] for idx in generated)


data = """ The meeting of Diogenes of Sinope and Alexander the Great is one of the most discussed anecdotes from philosophical history. Many versions of it exist. The most popular relate it as evidence of Diogenes' disregard for authority, wealth, and decorum.[1]Plutarch and Diogenes Laërtius report that Alexander and Diogenes died on the same day, in 323 BC.[2] Although this account is dubious (since neither man's date of death can be conclusively verified), the anecdote, and the relationship between the two people, has been the subject of many literary and artistic works over the centuries, from the writings of Diogenes Laërtius to David Pinski's 1930 dramatic reconstruction of the encounter, Aleḳsander un Dyogenes; including writings from the Middle Ages, several works of Henry Fielding, and possibly even Shakespeare's King Lear along the way. The literature and artwork influenced by this story are extensive.[3]Versions upon versions of the anecdote exist, with the origins of most appearing to be, either directly or indirectly, in the account of the meeting given by Plutarch, whose actual historicity has also been questioned.[3] Several of the embellished versions of the anecdote do not name either one or both of the protagonists, and some indeed substitute Socrates for Diogenes.[4]The original anecdoteAlexander und Diogenes by Lovis Corinth, 1894, at the Graphische Sammlung Albertinalithograph of the meeting of Alexander and Diogenes: Alexander, with an entourage of soldiers, standing over Diogenes sunbathing in the streetAlexander and Diogenes, lithograph illustration by Louis Loeb in Century Magazine, 1898According to legend, Alexander the Great came to visit the philosopher Diogenes of Sinope. Alexander wanted to fulfill a wish for Diogenes and asked him what he desired.[5] As told by Diogenes Laërtius, Diogenes replied, "Stand out of my light."[6] Plutarch provides a longer version of the story, which begins after Alexander arrives in Corinth:    Thereupon many statesmen and philosophers came to Alexander with their congratulations, and he expected that Diogenes of Sinope also, who was tarrying in Corinth, would do likewise. But since that philosopher took not the slightest notice of Alexander, and continued to enjoy his leisure in the suburb Craneion, Alexander went in person to see him, and he found him lying in the sun. Diogenes raised himself up a little when he saw so many people coming towards him, and fixed his eyes upon Alexander. And when that monarch addressed him with greetings, and asked if he wanted anything, "Yes," said Diogenes, "stand a little out of my sun."[7] It is said that Alexander was so struck by this, and admired so much the haughtiness and grandeur of the man who had nothing but scorn for him, that he said to his followers, who were laughing and jesting about the philosopher as they went away, "But truly, if I were not Alexander, I wish I were Diogenes."[8]There are many minor variants of what Diogenes is supposed to have replied to Alexander. According to Cicero, Diogenes answered Alexander with the words, "Now move at least a little out of the sun".[9] According to Valerius Maximus, Diogenes answered: "To this later, for now I just want you not to stand in the sun."[10] The statement by Alexander, "if I were not Alexander the Great, I would like to be Diogenes," also crops up in some other versions of the anecdote.[5]Arrian referred to the episode when recording the similar encounters of Indian philosophers with Alexander occurred during Alexander's campaigns in his book The Campaigns of Alexander.    When also in the Isthmus he met Diogenes of Sinope, lying in the sun, standing near him with his shield-bearing guards and foot Companions, he asked if he wanted anything. But Diogenes said that he wanted nothing else, except that he and his attendants would stand out of the sun. Alexander is said to have expressed his admiration of Diogenes's conduct. Thus it is evident that Alexander was not entirely destitute of better feelings; but he was the slave of his insatiable ambition. In his biography of Alexander, Robin Lane Fox[11] sets the encounter in 336, the only time Alexander was in Corinth. The Alexander of the story is not this great king, ruler of Greece and Asia, but the promising but brash 20-year-old son of Philip of Macedon, first proving his mettle in Greece. One of Diogenes' pupils, Onesicritus, later joined Alexander and will have been the original source of this story, embellished in the retelling, which appears in Ptolemy (14.2),[clarification needed] Arrian, (Anabasis Alexandri, 7.2.1) and "Plutarch" Moralia, 331.[12][13] The other major accounts of the tale are Cicero Tusculanae Disputationes 5.32.92; Valerius Maximus Dictorum factorumque memorabilium 4.3. ext. 4; Plutarch Alexander 14; and Diogenes Laërtius 6.32, 38, 60, and 68.[14]The historicity of the accounts by Plutarch and others has been questioned, not least by G. E. Lynch in his article on Diogenes in the Dictionary of Greek and Roman Biography and Mythology. Lynch points out the problem that Alexander did not have the title given to him until after he had left Greece, and considers this enough of a problem with the anecdote such that it (alongside the notion that Diogenes lived in a barrel) should be "banish[ed ...] from the domain of history". "[C]onsidering what rich materials so peculiar a person as Diogenes must have afforded for amusing stories," he continues, "we need not wonder if a few have come down to us of somewhat doubtful genuineness."[3][15] A. M. Pizzagalli suggests that the account has its origins in the meeting between Alexander and the Gymnosophists in India, and was handed down in Buddhist circles.[3][16]There are significant variations of fact amongst the accounts. Some have Diogenes and Alexander meeting at Corinth, some in Athens, and some at the Metroön. Further, as noted earlier, Diogenes Laërtius' rendition of the account is broken up into two parts. At 6.38 there is Alexander's request and Diogenes's "Stand out of my light!" reply. Alexander's aside to his followers is, however, at 6.32. At 6.68, D.L. has a third version of the anecdote, with Alexander responding that he is "a good thing" to an inquiry by Diogenes. At 6.60, D.L. has yet a fourth version, this time with the two exchanging introductions: "I am Alexander the great king." "I am Diogenes the dog."[3]In his Dialogues of the Dead (13), Lucian imagines a meeting between Alexander and Diogenes in the underworld. The philosopher once more punctures Alexander's pretensions and prescribes him a stiff draught from the water of Lethe.Interpretation by Dio ChrysostomDio Chrysostom, in his fourth oration on kingship,[17] ascribes a simple moral to the anecdote: people who are naturally outspoken and forthright respect others like themselves, whereas cowards regard such people as enemies. A good king will respect and tolerate the candour of a morally sincere critic (albeit that they must take care to determine which critics truly are sincere, and which are simply feigning sincerity), and Diogenes' remark to Alexander is a test of Diogenes. His bravery in risking offending Alexander, without knowing whether he would be tolerant of such behaviour beforehand, marks him as honest.[18]Interpretation by Peter SloterdijkAccording to Peter Sloterdijk, in his Critique of Cynical Reason, this is "perhaps the most well known anecdote from Greek antiquity, and not without justice". He states that "It demonstrates in one stroke what antiquity understands by philosophical wisdom – not so much a theoretical knowledge but rather an unerring sovereign spirit [...] The wise man [...] turns his back on the subjective principle of power, ambition, and the urge to be recognized. He is the first one who is uninhibited enough to say the truth to the prince. Diogenes' answer negates not only the desire for power, but the power of desire as such."[19]Interpretation by Samuel JohnsonSamuel Johnson wrote about this anecdote. Rather than relating it to Diogenes' cynicism, Johnson relates the story to time, relating the taking away of the sunlight by Alexander to the wasting of people's time by other people.[1] "But if the opportunities of beneficence be denied by fortune," wrote Johnson, "innocence should at least be vigilantly preserved. [...] Time [...] ought, above all other kinds of property, to be free from invasion; and yet there is no man who does not claim the power of wasting that time which is the right of others."[20]Modern interpretationsIn 2005, Ineke Sluiter analysed the proxemics of the encounter, observing that a common feature of the anecdotes was that Alexander approached Diogenes, reversing the usual stances of royalty and commoner in which the latter would be physically submissive. By such means, Diogenes communicated his cynical indifference to convention and status in a non-verbal way.[21]Medieval restructuring and reinterpretation16th century Alexandre et Diogène Urbino majolica in the Musée des Beaux-Arts de LyonThe anecdote was popular amongst medieval scholars, because of its mention in the writings of authors who were popular in that period: Cicero, Valerius Maximus, and Seneca. Valerius Maximus comments "Alexander Diogenem gradu suo diuitiis pellere temptat, celerius Darium armis" (transl. "Alexander tries to drive Diogenes with his stride, while Darius is faster with his arms") (4.3. ext. 4). Seneca says "multo potentior, multo locupletior fuit [Diogenes] omnia tunc possidente Alexandro: plus enim erat, quod hic nollet accipere quam quod ille posset dare." (transl. "[Diogenes] was much more powerful and much richer, since Alexander possessed everything at that time: for there was more that he would not accept than what he could give."), and adds "Alexander Macedonum rex gloriari solebat a nullo se beneficiis uictum." (transl. "Alexander, king of the Macedonians, used to boast that he was defeated by no favors.") (De beneficiis 5.4.3; 5.6.1).[22]These comments were widely reproduced. Philosophical thought in the Middle Ages agreed with Seneca in particular: Alexander, who boasted that no-one could surpass him when it came to liberality, was surpassed by Diogenes, who proved himself the better man by refusing to accept from Alexander everything except those things that Alexander could not give. Diogenes requests that Alexander return the sunshine to him, it being something that Alexander cannot give to him in the first place.[4][22]Diogenes' answer circulated as an aphorism in western Britain in the early Middle Ages, but it does not seem to have been understood or else had become completely divorced from the story. In the 9th-century dialogue De raris fabulis, "don't stand between me and the light" is the response of friend who is refusing a request for help because "other work engages me". In a later dialogue by Ælfric Bata, the aphorism is used to mean "stand a little further off", the advice to a younger monk of an elder using the latrine.[23]    Will is my man and my servant,    And evere hath ben and evere schal.    And thi will is thi principal,    And hath the lordschipe of thi witt,    So that thou cowthest nevere yit    Take o dai rests of thi labour;    Bot forto ben a conquerour    Of worldes good, which mai noght laste,    Thou hiest evere aliche faste,    Wher thou no reson hast to winne.Confessio Amantis (John Gower, 1390)[4]A different version of the anecdote, which included new material, changed the focus of the story. This version reached Europe through the Disciplina Clericalis and is also to be found in the Gesta Romanorum. In it, the incident of the sunlight is pushed into a subordinate position, with the main focus instead being upon Diogenes identifying Alexander as "the servant of his servant". In this modified anecdote, Diogenes states to Alexander that his (Diogenes') own will is subject to his reason, whereas Alexander's reason is subject to his will. Therefore, Alexander is the servant of his servant. The story of blocking the sunlight, in this version, is a brief introductory matter only; and, indeed, the tale is not even told as a meeting between Diogenes and Alexander, but as a meeting between Diogenes and Alexander's servants.[4][22]It was this latter form of the anecdote that became popular outside of scholarly circles in the Middle Ages. The former form, focused on the sunlight incident, was primarily confined to popularity amongst scholars.[22] John Gower presents this form of the anecdote in his Confessio Amantis. In the Confessio the meeting is a meeting of opposites. Alexander embodies a driven, restless and worldly conqueror, whereas Diogenes is the embodiment of philosophical virtue: rational control, patience and sufficiency. Alexander covets the world and laments the fact that he has no more to conquer ("al the world ne mai suffise To will which is noght reasonable" — Confessio Amantis III 2436–2437) whereas Diogenes is content with no more than the few necessities of nature.[4]Gower's re-telling of the anecdote names Diogenes and Alexander, and these are the two characters in most medieval versions of the anecdote. However, this is not the case for the Disciplina Clericalis nor for the Gesta Romanorum, this modified anecdote's earliest appearances. In the former, the meeting is between an unnamed king and Socrates; in the latter, it is between Socrates and Alexander. According to John David Burnley, this suggests that the anecdote, at least in this form, is meant to be an exemplar, rather than a literal truth. It does not matter precisely which characters are involved, as they are idealised forms rather than literal historical figures. They symbolize the conflict between a philosopher/critic and a king/conqueror, and it is the structure of the anecdote that is important, rather than the specific identities of the participants. Socrates is as good as Diogenes for this purpose, although Alexander is favoured as the king simply because by the Middle Ages he had already become the archetypical conqueror, and was considered the most famous one in history.[4]The encounter appears in numerous Elizabethan works such as John Lyly's play Campaspe. Shakespeare's play King Lear may have been intended to parody this when the King meets Edgar, son of Gloucester, dressed in rags and says "Let me talk with this philosopher".[3][24]Henry Fielding's DialogueEnglish Wikisource has original text related to this article:A Dialogue between Alexander the Great, and Diogenes the CynicHenry Fielding retells the anecdote as A Dialogue between Alexander the Great, and Diogenes the Cynic, printed in his Miscellanies in 1743.[25][26] Fielding's version of the story again uses Alexander as an idealistic representation of power and Diogenes as an idealistic representation of intellectual reflection. However, he portrays both men as fallible. Both are verbally adept, and engage one another, but both are dependent from the support of others for their weight of argument.[25] Fielding likes neither character, and in his version of the anecdote each serves to highlight the cruelty and meanness of the other.[27] The false greatness of the conqueror is shown opposed to the false greatness of the do-nothing philosopher, whose rhetoric is not carried through to action.[28]François Rabelais' allusionIn the Chapter XXX of François Rabelais' Pantagruel (c.1532), Pantagruel's tutor Epistemon had his head cut off after a battle. After he had his head reattached and was brought back to life, he recounts his experience of the damned in hell: "Their estate and condition of living is but only changed after a very strange manner; for I saw Alexander the Great there amending and patching on clouts upon old breeches and stockings, whereby he got but a very poor living."..."After this manner, those that had been great lords and ladies here, got but a poor scurvy wretched living there below. And, on the contrary, the philosophers and others, who in this world had been altogether indigent and wanting, were great lords there in their turn. I saw Diogenes there strut it out most pompously, and in great magnificence, with a rich purple gown on him, and a golden sceptre in his right hand. And, which is more, he would now and then make Alexander the Great mad, so enormously would he abuse him when he had not well patched his breeches; for he used to pay his skin with sound bastinadoes."Visual arts16th century Flemish paintingFlemish-German Renaissance painter Marten Van Valckenborg represented the anecdotic allegory from 330BC of Alexander, approaching the celebrated Cynic philosopher Diogenes. Ref Oil painting on wood, Alexander the Great visiting Diogenes, circa 1585, Private collection. Alexander the Great visiting Diogenes by Marten Van Valckenborg, 1585, in [Private collection] https://commons.wikimedia.org/wiki/File:Marten_Van_Valckenborg.jpgPuget's La rencontreLa rencontre d'Alexandre et de Diogène de Sinope by Pierre Paul Puget, 1680, in the Musée du LouvrePuget's bas relief, pictured at right, is widely regarded as a chef d'oeuvre.[29] Étienne Maurice Falconet described it as Puget's "sublime error".[30] Daniel Cady Eaton, art historian and professor of the History and Criticism of Art at Yale University, observed that the work is not in keeping with the anecdote, with Diogenes portrayed as a pitiable old man extending his arms and Alexander portrayed as mounted on a horse with a hand to his breast in mockery. The horses are too small for the riders, and the chain by which the dog is held is "big enough for a ship's anchor".[31] Eugène Delacroix wrote of the work:    If the great Puget had had as much of common sense as he had of the intensity and science which fill this work, he would have perceived before beginning that his subject was the strangest sculpture could choose. He forgot that in the mass of men, weapons, horses, and even edifices, he could not introduce the most essential actor; that is the sun's ray intercepted by Alexander; without which the composition has no sense.[31]Victor Duruy made the same point, writing:    Son bas-relief [...] est malgré la science qu'il y montra, une preuve de l'impuissance de la statuaire à rivaliser avec la peinture. Combien sont lourds ces nuages et ces drapeaux de marbre qui flotteraient si bien dans l'air libre d'un tableau! Et où est le principal acteur de cette scène, le rayon de soleil qu'Alexandre intercepte?[32]Others, such as Gonse, praised Puget:    I do not hesitate to proclaim the bas-relief of Alexandre de Diogène one of the most striking creations of modern sculpture. Everything that is most rare and most difficult in the art of sculpture are there united as by a miracle: concentrated plastic effect, play of lights and shadows, selections of plans, ease of modelling; nervous, fine, lively, and iridescent execution. What more can be said? There is not a secondary detail that is not treated with a marvelous assurance.[31]Alexander and Diogenes by Edwin Landseer, 1848, in the Tate collectionLandseer's Alexander and DiogenesEdwin Landseer's Alexander and Diogenes presents the encounter between the twain as between two dogs.[33] Alexander is a white bulldog with a military collar who looks down haughtily upon Diogenes, represented as a scruffy farrier's dog in a barrel.[34][35] Landseer was inspired to create the painting when he encountered two dogs in the street, one observing the other from within a barrel, and was reminded of the encounter between Alexander and Diogenes.[36] The painting in turn was to become the inspiration for the anthropomorphic dogs in Disney's 1955 film Lady and the Tramp.[37] Charles Darwin and Briton Rivière agreed with each other that the hair of the Alexander dog was inaccurately represented.[38]"""
tokens = []
for sentence in data.split():
    clean = re.sub(r"[,\!\?\(\):]", "", sentence.lower())
    if clean:
        tokens.append(clean)

special_tokens = {
    "<#START>": 0,
    "<#PAD>": 1,
    "<#UNKNOWN>": 2,
    "<#END>": 3,
}
vocab = sorted(set(tokens))
vocab_size = len(vocab) + len(special_tokens)
start_index = len(special_tokens)
word2idx = {word: idx + start_index for idx, word in enumerate(vocab)}
word2idx = {**special_tokens, **word2idx}
idx2word = {idx: word for word, idx in word2idx.items()}

token_ids = [word2idx.get(word, special_tokens["<#UNKNOWN>"]) for word in tokens]

X, y = [], []
for i in range(len(token_ids) - SEQUENCE_LEN):
    X.append(token_ids[i : i + SEQUENCE_LEN])
    y.append(token_ids[i + SEQUENCE_LEN])

X = np.array(X)
y = np.array(y)
rnn_model = RnnModel(SEQUENCE_LEN, vocab_size, RNN_LAYER_DIM, EMBEDDING_DIM)
rnn_model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE)

seed_text = [
    "the",
    "meeting",
    "of",
    "diogenes",
    "of",
    "sinope",
    "and",
    "alexander",
    "the",
    "great",
    "is",
    "one",
    "of",
    "the",
    "most",
    "discussed",
    "anecdotes",
    "from",
    "philosophical",
    "history",
]

seed_tokens = [word2idx.get(w, word2idx["<#UNKNOWN>"]) for w in seed_text]

print(rnn_model.generate(seed_tokens, idx2word, max_length=30))


Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0461 - loss: 6.7658
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0665 - loss: 5.9974
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0786 - loss: 5.6373
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0976 - loss: 5.3691
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1032 - loss: 5.1188
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.1097 - loss: 4.9038
Epoch 7/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.1486 - loss: 4.6841
Epoch 8/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1689 - loss: 4.5117
Epoch 9/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2136 - loss: 4.2240
Epoch 10/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2449 - loss: 4.0115
Epoch 11/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3102 - loss: 3.8226
Epoch 12/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step